In [1]:
ls /g/data/rt52/era5/pressure-levels/reanalysis/

cc/  ciwc/  clwc/  crwc/  cswc/  d/  o3/  pv/  q/  r/  t/  u/  v/  vo/  w/  z/


In [2]:
ls /g/data/rt52/era5/pressure-levels/reanalysis/u/1979

u_era5_oper_pl_19790101-19790131.nc  u_era5_oper_pl_19790701-19790731.nc
u_era5_oper_pl_19790201-19790228.nc  u_era5_oper_pl_19790801-19790831.nc
u_era5_oper_pl_19790301-19790331.nc  u_era5_oper_pl_19790901-19790930.nc
u_era5_oper_pl_19790401-19790430.nc  u_era5_oper_pl_19791001-19791031.nc
u_era5_oper_pl_19790501-19790531.nc  u_era5_oper_pl_19791101-19791130.nc
u_era5_oper_pl_19790601-19790630.nc  u_era5_oper_pl_19791201-19791231.nc


In [3]:
import track_to_ndarray as lib
import numpy as np
import os
import xarray

In [4]:
ds = xarray.open_mfdataset("/g/data/rt52/era5/pressure-levels/reanalysis/u/1979/u_era5_oper_pl_19791201-19791231.nc",
        chunks={ "time": 1, "latitude": 40, "longitude": 40 },
        combine='nested', concat_dim='time')
ds.u

,Array,Chunk
Bytes,106.47 GiB,231.25 kiB
Shape,"(744, 37, 721, 1440)","(1, 37, 40, 40)"
Count,508897 Tasks,508896 Chunks
Type,float32,numpy.ndarray


In [5]:
lib.sample_latlong_window(ds.u, 10, 0, -179, '1979-12-01T00:00:00').shape

(37, 40, 40)

In [6]:
import json
with open('../tracks/proc_tracks.json', 'r') as f:
    tracks = json.load(f)
print(tracks[list(tracks.keys())[0]])

{'iso_times': ['1978-11-19 12:00:00', '1978-11-19 15:00:00', '1978-11-19 18:00:00', '1978-11-19 21:00:00', '1978-11-20 00:00:00', '1978-11-20 03:00:00', '1978-11-20 06:00:00', '1978-11-20 09:00:00', '1978-11-20 12:00:00', '1978-11-20 15:00:00', '1978-11-20 18:00:00', '1978-11-20 21:00:00', '1978-11-21 00:00:00', '1978-11-21 03:00:00', '1978-11-21 06:00:00', '1978-11-21 09:00:00', '1978-11-21 12:00:00', '1978-11-21 15:00:00', '1978-11-21 18:00:00', '1978-11-21 21:00:00', '1978-11-22 00:00:00', '1978-11-22 03:00:00', '1978-11-22 06:00:00', '1978-11-22 09:00:00', '1978-11-22 12:00:00', '1978-11-22 15:00:00', '1978-11-22 18:00:00', '1978-11-22 21:00:00', '1978-11-23 00:00:00'], 'categories': ['-1', '-1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], 'coordinates': [[97.0, -10.2], [96.9062, -10.3518], [96.7707, -10.5129], [96.5511, -10.705], [96.2, -11.0], [95.6875, -11.4652], [95.0933, -11.9954], [94.

In [7]:
import dask
import traceback

# this prints out a million lines of 1 second progress bars, not very useful
from dask.diagnostics import ProgressBar
ProgressBar().register()

def error_tolerant(f, *args, **kwargs):
    try:
        return f(*args, **kwargs)
    except Exception:
        traceback.print_exc()

In [8]:
cyclone_thunks = []

for ssid in list(tracks.keys())[2000:2003]:
    track = tracks[ssid]
    cyclone_thunks.append(dask.delayed(
        error_tolerant(lib.track_to_ndarray, track['iso_times'], [(lat, long) for [long, lat] in track['coordinates']], degree_window=10)
    ))

cyclones = dask.compute(*cyclone_thunks)

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100

In [11]:
[cyclone.shape for cyclone in cyclones]

[(77, 4, 37, 40, 40), (33, 4, 37, 40, 40), (39, 4, 37, 40, 40)]